In [1]:
import hopsworks
import pandas as pd
#from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
import numpy as np
import seaborn as sns
from matplotlib import pyplot
from hsml.schema import Schema
from hsml.model_schema import ModelSchema
import joblib
import os
from sklearn.ensemble import RandomForestRegressor

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split


/home/codespace/.python/current/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# You have to set the environment variable 'HOPSWORKS_API_KEY' for login to succeed
project = hopsworks.login()
fs = project.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/194714
Connected. Call `.close()` to terminate connection gracefully.


#### Get or Create a new feature view

In [3]:
# The feature view is the input set of features for your model. The features can come from different feature groups.    
# You can select features from different feature groups and join them together to create a feature view
flight_fg = fs.get_feature_group(name="flight_data_v3", version=1)
query = flight_fg.select_all()
feature_view = fs.get_or_create_feature_view(name="flight_data_v3",
                                  version=1,
                                  description="Read from Flight Delay dataset",
                                  labels=["dep_delay_new"],
                                  query=query)

#### Create a new train test split

In [12]:
# You can read training data, randomly split into train/test sets of features (X) and labels (y)        
X_train, X_test, y_train, y_test = feature_view.train_test_split(test_size=0.2)

Finished: Reading data from Hopsworks, using ArrowFlight (16.91s) 


#### Read from existing train test split

In [11]:
#flight_fg = fs.get_feature_group(name="flight_data", version=1)
#query = flight_fg.select_all()
feature_view = fs.get_feature_view(name="flight_data_v3",version=1)
# get a training dataset with train and test splits
X_train, X_test, y_train, y_test = feature_view.get_train_test_split(training_dataset_version=6)


KeyboardInterrupt



In [4]:
X_train

,month,day_of_month,day_of_week,origin_airport_id,origin_wac,dest_airport_id,dest_wac,crs_dep_time,crs_arr_time,hourlydewpointtemperature,hourlydrybulbtemperature,hourlyprecipitation,hourlyrelativehumidity,hourlystationpressure,hourlyvisibility,hourlywinddirection,hourlywindgustspeed,hourlywindspeed
0,3,5,5,12,33,11,33,1315,1426,12.8,25.6,0.0,45.0,1013.88,16.00,80,0.0,4.0
1,4,21,3,1,36,11,33,900,1044,6.7,21.7,0.0,38.0,982.39,16.09,270,11.0,6.0
2,8,30,2,9,85,12,33,2327,715,6.1,33.3,0.0,18.0,936.34,16.09,210,0.0,3.0
3,7,16,6,16,91,2,82,1100,1445,11.7,22.2,0.0,52.0,1012.53,16.09,310,0.0,8.0
4,2,2,3,16,91,4,21,1450,2301,1.1,15.6,0.0,38.0,1020.32,16.09,360,0.0,8.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1662951,5,16,7,14,23,0,34,1340,1555,3.9,21.1,0.0,32.0,1022.35,16.00,230,0.0,3.0
1662952,5,18,3,8,22,5,33,2035,2353,10.6,14.4,0.0,78.0,1011.51,16.09,190,0.0,7.0
1662953,6,30,4,7,74,13,41,630,914,22.2,23.9,0.0,90.0,1012.87,16.00,20,0.0,3.0
1662954,11,5,6,10,91,16,91,1049,1217,5.0,18.9,0.0,40.0,1006.10,16.09,250,0.0,4.0


In [5]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1330364 entries, 0 to 1662955
Data columns (total 18 columns):
 #   Column                     Non-Null Count    Dtype  
---  ------                     --------------    -----  
 0   month                      1330364 non-null  int64  
 1   day_of_month               1330364 non-null  int64  
 2   day_of_week                1330364 non-null  int64  
 3   origin_airport_id          1330364 non-null  int64  
 4   origin_wac                 1330364 non-null  int64  
 5   dest_airport_id            1330364 non-null  int64  
 6   dest_wac                   1330364 non-null  int64  
 7   crs_dep_time               1330364 non-null  int64  
 8   crs_arr_time               1330364 non-null  int64  
 9   hourlydewpointtemperature  1330364 non-null  float64
 10  hourlydrybulbtemperature   1330364 non-null  float64
 11  hourlyprecipitation        1330364 non-null  float64
 12  hourlyrelativehumidity     1330364 non-null  float64
 13  hourlystationpres

In [6]:
y_train

,dep_delay_new
0,0.0
1,0.0
2,0.0
3,8.0
4,0.0
...,...
1662951,0.0
1662952,24.0
1662953,0.0
1662954,21.0


In [7]:
y_train.values.ravel()

array([ 0.,  0.,  0., ...,  0., 21.,  6.])

#### Random Forest Regressor Model

In [ ]:
# Initialize the Random Forest classifier
model_RF = RandomForestRegressor(max_depth=10,random_state=1)  # You can adjust parameters here

# Train the classifier
model_RF.fit(X_train, y_train)
# Predict labels for the test set
y_pred = model_RF.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
# Calculate RMSE
rmse = np.sqrt(mse)

# Calculate MAE
mae = mean_absolute_error(y_test, y_pred)

# Calculate R-squared
r2 = r2_score(y_test, y_pred)

print(f"RMSE: {rmse}")
print(f"MAE: {mae}")
print(f"R-squared: {r2}")
print(f"Mean Squared Error (MSE): {mse}")

In [11]:
y_train_np=y_train.values.ravel()
X_train_np=X_train.values
X_test_np = X_test.values
y_test_np=y_test.values.ravel()

In [ ]:
# Initialize the Random Forest classifier
model_RF_np = RandomForestRegressor(max_depth=10,random_state=1)  # You can adjust parameters here

# Train the classifier
model_RF_np.fit(X_train_np, y_train_np)
# Predict labels for the test set
y_pred_np = model_RF_np.predict(X_test_np)

mse = mean_squared_error(y_test_np, y_pred_np)
# Calculate RMSE
rmse = np.sqrt(mse)

# Calculate MAE
mae = mean_absolute_error(y_test_np, y_pred_np)

# Calculate R-squared
r2 = r2_score(y_test_np, y_pred_np)

print(f"RMSE: {rmse}")
print(f"MAE: {mae}")
print(f"R-squared: {r2}")
print(f"Mean Squared Error (MSE): {mse}")

#### Linear Regression

In [ ]:
# Initialize the Linear Regression model
model_lin_reg = LinearRegression()

# Fit the model on the training data
model_lin_reg.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model_lin_reg.predict(X_test)

# Calculate the Mean Squared Error (MSE) on test set

mse = mean_squared_error(y_test, y_pred)
# Calculate RMSE
rmse = np.sqrt(mse)

# Calculate MAE
mae = mean_absolute_error(y_test, y_pred)

# Calculate R-squared
r2 = r2_score(y_test, y_pred)

print(f"RMSE: {rmse}")
print(f"MAE: {mae}")
print(f"R-squared: {r2}")
print(f"Mean Squared Error (MSE): {mse}")

#### Neural Network

In [4]:
# Convert data to PyTorch tensors
X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test.values, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32)

In [5]:
# Normalize features (optional but often recommended for neural networks)
# outputs np arrays
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_tensor)
X_test_scaled = scaler.transform(X_test_tensor)

In [6]:
# Convert the scaled NumPy arrays back to PyTorch tensors
X_train_tensor = torch.tensor(X_train_scaled, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32)

In [7]:
# Convert to PyTorch Dataset
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

In [8]:
class NeuralNetwork(nn.Module):
    def __init__(self, input_size):
        super(NeuralNetwork, self).__init__()
        self.fc1 = nn.Linear(input_size, 128)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.2)
        self.fc2 = nn.Linear(128, 64)
        self.output = nn.Linear(64, 1)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.output(x)
        return x

In [ ]:
# Initialize the model
input_size = X_train_tensor.shape[1]
model = NeuralNetwork(input_size)

# Define loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

# Define data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32)

# Train the model
num_epochs = 25
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, targets in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    
    #convert to rmse
    train_loss = running_loss / len(train_loader)
    train_rmse = torch.sqrt(torch.tensor(train_loss))
    print(f"Epoch {epoch+1}/{num_epochs}, Train Loss MSE: {train_loss}, Train Loss RMSE: {train_rmse}")
model.eval()
test_loss = 0.0
with torch.no_grad():
    for inputs, targets in test_loader:
        outputs = model(inputs)
        test_loss += criterion(outputs, targets).item()
    test_loss /= len(test_loader)

test_rmse = torch.sqrt(torch.tensor(test_loss))
print(f"Test Loss MSE: {test_loss}, Test RMSE: {test_rmse}")

#### Save to Hopsworks Model Registry 

In [17]:
# We will now upload our model to the Hopsworks Model Registry. First get an object for the model registry.
mr = project.get_model_registry()

# The contents of the 'iris_model' directory will be saved to the model registry. Create the dir, first.
model_dir="flight delay model"
if os.path.isdir(model_dir) == False:
    os.mkdir(model_dir)

# Save both our model and the confusion matrix to 'model_dir', whose contents will be uploaded to the model registry
joblib.dump(model, model_dir + "/flight_delay_model.pkl")
#fig.savefig(model_dir + "/confusion_matrix.png")    

# Specify the schema of the model's input/output using the features (X_train) and labels (y_train)
#input_schema = Schema(X_train_tensor)
#output_schema = Schema(y_train_tensor)
#model_schema = ModelSchema(input_schema, output_schema)

# Create an entry in the model registry that includes the model's name, desc, metrics
fligt_delay_model = mr.python.create_model(
    name="flight_delay_model", 
    #metrics={"accuracy" : metrics['accuracy']},
    metrics={"RMSE" : test_rmse.item()},
    #model_schema=model_schema,
    description="Flight Delay Predictor no schema"
)

# Upload the model to the model registry, including all files in 'model_dir'
fligt_delay_model.save(model_dir + "/flight_delay_model.pkl")

Connected. Call `.close()` to terminate connection gracefully.


Uploading: 100.000%|██████████| 46277/46277 elapsed<00:03 remaining<00:001,  3.02it/s]
Model export complete: 100%|██████████| 6/6 [00:09<00:00,  1.61s/it]                   

Model created, explore it at https://c.app.hopsworks.ai:443/p/194714/models/flight_delay_model/2


Model(name: 'flight_delay_model', version: 2)

### Example showing how to load the NN model from Model Registry and how to use it for inference

Class definition needed due to how python loads saved pytorch models. The class definition is used to load the pickle model. If the pytorch model was saved as a state dict it could also be loaded using state_dict without defining class definition

In [18]:
class NeuralNetwork(nn.Module):
    def __init__(self, input_size):
        super(NeuralNetwork, self).__init__()
        self.fc1 = nn.Linear(input_size, 128)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.2)
        self.fc2 = nn.Linear(128, 64)
        self.output = nn.Linear(64, 1)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.output(x)
        return x

mr = project.get_model_registry()
model = mr.get_model("flight_delay_model", version=2)
model_dir = model.download()
model = joblib.load(model_dir + "/flight_delay_model.pkl")


Connected. Call `.close()` to terminate connection gracefully.


In [22]:
#Data sample used for inference
i_test=3
a=X_train.iloc[i_test]
a=torch.tensor(a, dtype=torch.float32)

#Fit scaler to how it was fitted during training
scaler = StandardScaler()
X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32)
X_train_scaled = scaler.fit_transform(X_train_tensor)
X_train_tensor = torch.tensor(X_train_scaled, dtype=torch.float32)

#transform data sample
a=scaler.transform(a.reshape(1, -1))
print(a)
print(X_train_tensor[i_test])

# transform np array to torch tensor
a_tensor=torch.tensor(a, dtype=torch.float32)

print(a_tensor)
print(model(a_tensor))
print(y_train.iloc[i_test])

[[ 0.06875949  0.02368773  1.0026548   1.61362846  1.38625815 -1.17068383
   1.04279966 -0.47209716 -0.08938165  0.26542597  0.43838145 -0.10021307
  -0.40318245  0.50681242  0.33170667  1.22564572 -0.35572269  1.58065028]]
tensor([ 0.0688,  0.0237,  1.0027,  1.6136,  1.3863, -1.1707,  1.0428, -0.4721,
        -0.0894,  0.2654,  0.4384, -0.1002, -0.4032,  0.5068,  0.3317,  1.2256,
        -0.3557,  1.5807])
tensor([[ 0.0688,  0.0237,  1.0027,  1.6136,  1.3863, -1.1707,  1.0428, -0.4721,
         -0.0894,  0.2654,  0.4384, -0.1002, -0.4032,  0.5068,  0.3317,  1.2256,
         -0.3557,  1.5807]])
tensor([[11.6275]], grad_fn=<AddmmBackward0>)
dep_delay_new    8.0
Name: 3, dtype: float64
